# TigerGraph and Cytoscape

## Set up your TigerGraph Graph

### Start Your TigerGraph Solution

Go to https://tgcloud.io/ and if your solution says "Stop" next to it, press the Application Operations and select "Start" from the dropdown.

### Connect to TigerGraph

In [ ]:
!pip install pyTigerGraph

In [ ]:
import pyTigerGraph as tg 

conn = tg.TigerGraphConnection(host="https://spotifygraph.i.tgcloud.io/", password="tigergraph", graphname="SpotifyGraph")
conn.apiToken = conn.getToken(conn.createSecret())

In [ ]:
# NOTE: ONLY UNCOMMENT FOR EMERGENCIES!! IT WILL DELETE EVERYTHING FROM YOUR GRAPH!

# print(conn.gsql('''DROP ALL'''))

## TigerGraph and Cytoscape with Plotly Dash

### Import the Libraries

In [ ]:
!pip install -q jupyter-dash
!pip install dash_cytoscape

import dash_cytoscape as cyto
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd 

### Create the App

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("CytoTG + Spotify"),
    html.H2("Style"),
    dcc.Dropdown(
        id = "layout",
        options=[
            {"value": "preset", "label": "preset"},
            {"value": "random", "label": "random"},
            {"value": "grid", "label": "grid"},
            {"value": "circle", "label": "circle"},
            {"value": "concentric", "label": "concentric"},
            {"value": "breadthfirst", "label": "breadthfirst"},
            {"value": "cose", "label": "cose"},
        ],
        value="circle",
        clearable=False
    ),
    html.H2("Data"),
    dcc.Dropdown(
        id = "opt",
        options=[
            {'value': "Artist", 'label': "Artist"},
            {'value': "Playlist", 'label': "Playlist"},
            {'value': 'Genre', 'label': 'Genre'},
            {'value': 'Song', 'label': 'Song'}
        ],
        value="Playlist",
        clearable=False
    ),
    dcc.Dropdown(
        id = "artist",
        value="Altar",
        clearable=False
    ),
    cyto.Cytoscape(
        id = "cyto",
        layout={
            'name': 'circle'
        },
        style={'width': '100%', 'height': '400px'}
    ),
    cyto.Cytoscape(
        id = "scape",
        layout={
            'name': 'circle'
        },
        style={'width': '100%', 'height': '400px'}
    ),
])    

### Layout Callbacks

In [ ]:
# Updates the first cytoscape graph based on a change in layout

@app.callback(
    Output('cyto', 'layout'),
    Input('layout', 'value')
)
def update_graph(lay):
    return {"name": lay}

# Updates the second cytoscape graph based on a change in layout

@app.callback(
    Output('scape', 'layout'),
    Input('layout', 'value')
)
def update_graph(lay):
    return {"name": lay}

In [ ]:
# Updates the dropdowns

@app.callback(
    Output('artist', 'options'),
    Input('opt', 'value'),
)
def update_graph(opt):
    if opt == "Artist":
        artists = conn.runInstalledQuery("getArtists")
        artists = [{"label": i["attributes"]["name"], "value": i["attributes"]["name"]} for i in artists[0]["Seed"]]
        return artists
    elif opt == "Playlist":
        playlists = conn.runInstalledQuery("getPlaylists")
        playlists = [{"label": i["attributes"]["name"], "value": i["attributes"]["name"]} for i in playlists[0]["Seed"]]
        return playlists
    elif opt == "Genre":
        genres = conn.runInstalledQuery("getGenres")
        genres = [{"label": i["attributes"]["name"], "value": i["attributes"]["name"]} for i in genres[0]["Seed"]]
        return genres
    elif opt == "Song":
        songs = conn.runInstalledQuery("getSongs")
        songs = [{"label": i["attributes"]["name"], "value": i["attributes"]["name"]} for i in songs[0]["Seed"]]
        return songs

### Cytoscape Callbacks

In [ ]:
# Updates the first cytoscape graph elements

@app.callback(
    Output('cyto', 'elements'),
    Input('artist', 'value'),
    Input('opt', 'value'),
)
def update_graph(art, opt):
    if opt == "Artist":
        res = conn.runInstalledQuery("getSongsByArtist", {"artist": art})[0]["@@songs"] # Runs query
        cyto_data = [{'data': {'id': art, 'label': art, 'classes': 'artist'}}] # Adds artist vertex
        for i in res:
            cyto_data.append({"data": {'id': i['name'], 'label': i['name'], 'classes': 'song'}}) # Adds songs vertices
            cyto_data.append({'data': {'source': art, 'target': i['name']}}) # Adds edges
        return cyto_data
    elif opt == "Playlist": 
        res = conn.runInstalledQuery("getSongsByPlaylist", {"playlist": art})[0]["@@songs"] # Runs query
        cyto_data = [{'data': {'id': art, 'label': art, 'classes': 'playlist'}}] # Adds artist vertex
        for i in res:
            cyto_data.append({"data": {'id': i['name'], 'label': i['name'], 'classes': 'song'}}) # Adds songs vertices
            cyto_data.append({'data': {'source': art, 'target': i['name']}}) # Adds edges
        return cyto_data
    elif opt == "Genre": 
        res = conn.runInstalledQuery("getSongsByGenre", {"genre": art})[0]["@@songs"] # Runs query
        cyto_data = [{'data': {'id': art, 'label': art, 'classes': 'genre'}}] # Adds genre vertex
        for i in res:
            cyto_data.append({"data": {'id': i['name'], 'label': i['name'], 'classes': 'song'}}) # Adds songs vertices
            cyto_data.append({'data': {'source': art, 'target': i['name']}}) # Adds edges
        return cyto_data
    elif opt == "Song": 
        res = conn.runInstalledQuery("getVerticesFromSongs", {"song_name": art})[0]["connectedVals"] # Runs query
        cyto_data = [{'data': {'id': art, 'label': art, 'classes': 'song'}}] # Adds song vertex
        for i in res:
            if i['vertex_type'] == 'genre':
                cyto_data.append({"data": {'id': i['name'], 'label': i['name'], 'classes': 'genre'}}) # Adds genres vertices
            if i['vertex_type'] == 'artist':
                cyto_data.append({"data": {'id': i['name'], 'label': i['name'], 'classes': 'artist'}}) # Adds artist vertices
            if i['vertex_type'] == 'playlist':
                cyto_data.append({"data": {'id': i['name'], 'label': i['name'], 'classes': 'playlist'}}) # Adds playlist vertices
            cyto_data.append({'data': {'source': art, 'target': i['name']}}) # Adds edges
        return cyto_data
    else:
        return []


In [ ]:
# Updates the second cytoscape graph elements


@app.callback(
    Output('scape', 'elements'),
    Input('cyto', 'tapNodeData'),
)
def update_graph(main_data):
    if (main_data):
        art = main_data["id"]
        opt = main_data["classes"]
        if opt == "artis":
            res = conn.runInstalledQuery("getSongsByArtist", {"artist": art})[0]["@@songs"] # Runs query
            cyto_data = [{'data': {'id': art, 'label': art}}] # Adds artist vertex
            for i in res:
                cyto_data.append({"data": {'id': i['name'], 'label': i['name'], 'classes': 'song'}}) # Adds songs vertices
                cyto_data.append({'data': {'source': art, 'target': i['name']}}) # Adds songs edges
            return cyto_data
        elif opt == "playlist": 
            res = conn.runInstalledQuery("getSongsByPlaylist", {"playlist": art})[0]["@@songs"] # Runs query
            cyto_data = [{'data': {'id': art, 'label': art}}] # Adds artist vertex
            for i in res:
                cyto_data.append({"data": {'id': i['name'], 'label': i['name'], 'classes': 'song'}}) # Adds songs vertices
                cyto_data.append({'data': {'source': art, 'target': i['name']}}) # Adds songs edges
            return cyto_data
        elif opt == "genre": 
          res = conn.runInstalledQuery("getSongsByGenre", {"genre": art})[0]["@@songs"] # Runs query
          cyto_data = [{'data': {'id': art, 'label': art, 'classes': 'genre'}}] # Adds genre vertex
          for i in res:
              cyto_data.append({"data": {'id': i['name'], 'label': i['name'], 'classes': 'song'}}) # Adds songs vertices
              cyto_data.append({'data': {'source': art, 'target': i['name']}}) # Adds songs edges
          return cyto_data
        elif opt == "song": 
            res = conn.runInstalledQuery("getVerticesFromSongs", {"song_name": art})[0]["connectedVals"] # Runs query
            cyto_data = [{'data': {'id': art, 'label': art, 'classes': 'song'}}] # Adds artist vertex
            for i in res:
                if i['vertex_type'] == 'genre':
                    cyto_data.append({"data": {'id': i['name'], 'label': i['name'], 'classes': 'genre'}}) # Adds songs vertices
                if i['vertex_type'] == 'artist':
                    cyto_data.append({"data": {'id': i['name'], 'label': i['name'], 'classes': 'artist'}}) # Adds songs vertices
                if i['vertex_type'] == 'playlist':
                    cyto_data.append({"data": {'id': i['name'], 'label': i['name'], 'classes': 'playlist'}}) # Adds songs vertices
                cyto_data.append({'data': {'source': art, 'target': i['name']}}) # Adds songs edges
            return cyto_data
        else: 
            return []
    else:
        return []


### Run the App

In [ ]:
app.run_server(mode='external')

Dash app running on:


<IPython.core.display.Javascript object>

## Congrats!

Congratulations! You have now visualised the data in your TigerGraph graph database with cytoscape! 


Want to learn more? Connect with other developers? 
* Join the [TigerGraph Discord](https://discord.gg/gRHWBZNpxW)! 
* Check out the [TigerGraph Developers' Portal](https://developers.tigergraph.com/) for more cool TigerGraph tools!
* Finally, create your own project like this and submit it to the [Community Contribution Program](https://www.tigergraph.com/community-contribution/)!

